## Ansatze and variational forms

At the heart of all variational algorithms is the key idea of analyzing the differences between states, conveniently related through some well behaved mapping (e.g. continuous, differentiable) from a set of parameters or _variables_ — hence the name. 

First, we'll explore how to construct parametrized circuits by hand. We'll use parametrized circuits define a *variational form* to represent a collection of parametrized states for our variational algorithm to explore. Then, we'll construct our *ansatz* by applying this variational form onto our reference state.

We'll also explore how to tradeoff speed vs accuracy while exploring this search space.

![Ansatz](ansatz.png)

## Parameterized Quantum Circuits

Variational algorithms work by exploring and comparing a range of _quantum states_ $|\psi(\vec{\theta})\rangle$, depending on a (finite) set of $k$ parameters $\vec{\theta} = (\theta^0,...,\theta^{k-1})$. We can prepare these states with a _parametrized_ quantum circuit, where gates are defined with tunable parameters. We can prepare this parametrized circuit without having to bind specific angles yet:

In [ ]:
from qiskit.circuit import QuantumCircuit, Parameter

theta = Parameter("θ")

qc = QuantumCircuit(3)
qc.rx(theta, 0)
qc.cx(0,1)
qc.x(2)

qc.draw("mpl")

In [ ]:
from math import pi

angle_list = [pi/3, pi/2]
circuits = [qc.bind_parameters({theta: angle}) for angle in angle_list]

for circuit in circuits:
    display(circuit.draw("mpl"))

## Variational Form and Ansatz

To begin iteratively optimizing from our reference state $|\rho\rangle$ to the target state $|\psi(\vec\theta)\rangle$, we must define a *variational form* $U_V(\vec\theta)$ to represent a collection of parametrized states for our variational algorithm to explore:

$$
\begin{aligned}
|0\rangle \xrightarrow{U_R} U_R|0\rangle

& = |\rho\rangle \xrightarrow{U_V(\vec{\theta})} U_A(\vec{\theta})|0\rangle \\[1mm]

& = U_V(\vec{\theta})U_R|0\rangle \\[1mm]

& = U_V(\vec{\theta})|\rho\rangle \\[1mm]

& = |\psi(\vec{\theta})\rangle \\[1mm]

\end{aligned}
$$

Notice that the parametrized state depends both on the reference state $|\rho\rangle$, which does not depend on any parameters, as well as on the variational form $U_V(\vec{\theta})$, which always depends on parameters. We call the combination of these two halves an _ansatz_: $U_A(\vec\theta) := U_V(\vec\theta)U_R$.

As we construct our ansatz to represent a collection of parametrized states for our variational algorithm to explore, we'll realise an important issue: dimensionality. An $n$-qubit system (i.e. _Hilbert space_) has a vast amount of distinct quantum states in the configuration space. We would require an unwieldy large number of parameters to fully explore it. Quantitatively, its dimensionality is $D = 2^{2n}$. To make matters worse, the runtime complexity of search algorithms —and others alike— grows exponentially with this dimensionality, a phenomenon often referred to in the literature as the _curse of dimensionality_.

To counter this setback, it is common practice to impose some (reasonable) constraints on the variational form such that only the most relevant states are explored. Finding efficient truncated ansatz is an active area of research, but we'll cover two common designs.

## Heuristic ansatze and tradeoffs

If you don't have any information about your particular problem that can help restrict the dimensionality, you can try an arbitrary family of parameterized circuits with less than $2^{2n}$ parameters. We have a few tradeoffs to note:

- **Speed**: This will reduce the search space
- **Accuracy**: Reducing the space could risk excluding the actual solution to the problem, leading to suboptimal solutions

There is a fundamental tradeoff between quality (or even solvability) and speed: the more parameters, the more likely you are to find a precise result, but the longer it will take to run the algorithm.

### N-local circuits

One of the most widely used examples of heuristic ansatze are the [N-local circuits](https://qiskit.org/documentation/apidoc/circuit_library.html#n-local-circuits), due to a few reasons:

- **Efficient implementation**: The N-local ansatz is typically composed of simple, local gates that can be implemented efficiently on a quantum computer, using a small number of physical qubits. This makes it easier to construct and optimize quantum circuits.
- **Captures important correlations**: The N-local ansatz can capture important correlations between the qubits in a quantum system, even with a small number of gates. This is because the local gates can act on neighboring qubits and create entanglement between them, which can be important for simulating complex quantum systems.

These circuits consist of rotation and entanglement layers that are repeated alternatively one or more times as follows:

- Each layer is formed by gates of size at most $N$, where $N$ has to be lower than the number of qubits.
- For a rotation layer, the gates are stacked on top of each other. We can use standard rotation operations, such as [`RX`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RXGate.html) or [`CRZ`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CRZGate.html)
- For an entanglement layer, we can use gates like [`Toffoli` gates](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CCXGate.html) or [`CX`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html#qiskit.circuit.library.CXGate) with an entanglement strategy.
- Both types of layers can be parameterized or not, but at least one of them needs to contain parameters. Otherwise, without at least one parameter, there wouldn't be any *variations*!
- Optionally, an extra rotation layer is added to the end of the circuit.

For example, let's create a $5$-qubit [`NLocal`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.NLocal.html) circuit with rotation blocks formed by [`RX`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RXGate.html) and [`CRZ`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CRZGate.html) gates, entanglement blocks formed by [`Toffoli` gates](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CCXGate.html) that act on qubits $[0,1,2]$, $[0,2,3]$, $[4,2,1]$ and $[3,1,0]$ and $2$ repetitions of each layer.

In [ ]:
from qiskit.circuit.library import NLocal, CCXGate, CRZGate, RXGate
from qiskit.circuit import Parameter

theta = Parameter('θ')
qc = NLocal(num_qubits=5, rotation_blocks=[RXGate(theta), CRZGate(theta)], entanglement_blocks=CCXGate(), entanglement=[[0,1,2],[0,2,3],[4,2,1],[3,1,0]], reps=2, insert_barriers=True)
qc.decompose().draw("mpl")

In the above example, the biggest gate in this case is the Toffoli, acting on $3$ qubits. That means this circuit is $3$-local. The most common type of $N$-local circuits are [2-local](https://qiskit.org/documentation/stubs/qiskit.circuit.library.TwoLocal.html) with single-qubit rotation gates and $2$-qubit entanglement gates. 

Let's create a $2$-local circuit with Qiskit's [`TwoLocal`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.TwoLocal.html) class. The syntax is the same as `NLocal`'s but there are some differences. For instance, most gates, like the [`RX`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RXGate.html) and [`RZ`](https://qiskit.org/documentation/stubs/qiskit.circuit.library.RZGate.html) and [`CNOT` (`CX`)](https://qiskit.org/documentation/stubs/qiskit.circuit.library.CXGate.html) can be passed as strings without importing the gates or creating a [`Parameter`](https://qiskit.org/documentation/stubs/qiskit.circuit.Parameter.html) instance.

In [ ]:
from qiskit.circuit.library import TwoLocal

qc = TwoLocal(num_qubits = 5, rotation_blocks=['rx', 'rz'], entanglement_blocks='cx', entanglement="linear", reps=2, insert_barriers=True)
qc.decompose().draw("mpl")

In this case we used the `linear` entanglement distribution, where each qubit is entangled with the next. For other default strategies, check [the documentation](https://qiskit.org/documentation/stubs/qiskit.circuit.library.TwoLocal.html). 

## Problem-specific ansatze

If you have some theoretical knowledge about your problem that helps you restrict your circuit search space to a specific type, you can use it to gain speed without losing accuracy.

### Quantum Chemistry

For example, in Chemistry the [_Unitary Coupled-Cluster (UCC)_](https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.circuit.library.UCC.html#qiskit_nature.second_q.circuit.library.UCC) ansatz is used to solve the electronic structure problem while the [_Unitary Vibrational Coupled-Cluster (UVCC)_](https://qiskit.org/documentation/nature/stubs/qiskit_nature.second_q.circuit.library.UVCC.html#qiskit_nature.second_q.circuit.library.UVCC) ansatz is used to solve the vibrational structure problem.

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

h2_problem = driver.run()

converter = QubitConverter(JordanWignerMapper())

h2_reference_state = HartreeFock(
    h2_problem.num_spatial_orbitals,
    h2_problem.num_particles,
    converter
)

ansatz = UCCSD(
    h2_problem.num_spatial_orbitals,
    h2_problem.num_particles,
    converter,
    initial_state=h2_reference_state,
)

ansatz.decompose().decompose().draw('mpl')

Notice our original Hartree-Fock reference state in the output circuit. The subsequent blocks are the variational form. Combined, this ansatz should balance the speed and accuracy to solve the electronic structure problem

### Quantum Machine Learning 

Our original [variational quantum classifier (VQC)](https://learn.qiskit.org/course/machine-learning/variational-classification) reference state used a [ZFeatureMap](https://qiskit.org/documentation/stubs/qiskit.circuit.library.ZFeatureMap.html) to represent a data point from the training dataset. We can load classical data as an input, translate it into a quantum state, and limit our search space using an ansatz composed of parametrized circuits with trainable weights.

In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit.library import ZFeatureMap

z_feature_map_reference = ZFeatureMap(3, reps=3, insert_barriers=True)
z_feature_map_reference.decompose().draw('mpl')

num_qubits = 3

vcq_ansatz = RealAmplitudes(num_qubits=num_qubits, reps=1)

vqc_ansatz = QuantumCircuit(num_qubits)
vqc_ansatz.compose(z_feature_map_reference, inplace=True)
vqc_ansatz.compose(ansatz, inplace=True)

vqc_ansatz.decompose().draw('mpl')

Notice our original `ZFeatureMap` reference state in the output circuit. We can pass input data into the reference state, and then use the subsequent `RealAmplitudes` ansatz to pass our tuned our weights and limit the search space to converge.

With this lesson, you learned how to define your search space with a variational form:

- Prepare states with a _parametrized_ quantum circuit, where gates are defined with tunable parameters.
- How to construct ansatze that tradeoff speed vs accuracy
- Heuristic ansatze 
- Problem-specific ansatze

Our high-level variational circuit looks as follows:

![Ansatz Circuit](circuit_ansatz.png)

For each variational parameter $\vec\theta$, a different quantum state will be produced. To find the optimial parameters, we need to define a problem-specific _cost function_ to iteratively update our ansatz's parameters.